## PROCESAMIENTO DEL DATA SET DE VOLCANES

En primera medida hago los imports necesarios para el correcto funcionamiento del programa.

In [6]:
import csv
import os
import json

Defino mis funciones para el procesamiento de datos del archivo original (no las explico en markdown porque estan todas documentadas).

In [7]:
def generarNuevo(reader,lista_bolleanos,lista_nombres,lista_traduccion, dicc):
    """Recorro una sola vez el archivo y creo 3 listas que vendrian a ser cada
    columna del archivo original


    Args:
        reader (): archivo original
        lista_bolleanos (list): lista de tuplas de las columnas de flags
        lista_nombres (list): listas que tendran este formato = "nombre(pais)"
        lista_traduccion (list): lista de nombres traducidos
        dicc (dict): diccionario con todos los nombres de volcanes traducidos que se tomo desde un archivo

    Returns:
        list: 3 listas antes mencionadas en este orden, lista_boleanos, lista_nombres, lista_de_traducciones
    """
    for l in reader:
        if l[3] == '' and l[4] == '': #LOS DOS VACIOS
            lista_bolleanos.append(['F', 'F'])
        elif l[3] == '' and l[4] != '': #PRIMERO = VACIO SEGUNDO = ALGO
            lista_bolleanos.append(['F', 'T'])
        elif l[3] != '' and l[4] == '':#PRIMERO = ALGO SEGUNDO = VACIO
            lista_bolleanos.append(['T', 'F'])
        elif l[3] != '' and l[4] != '':#PRIMERO = LLENO SEGUNDO = LLENO
            lista_bolleanos.append(['T', 'T'])

        lista_nombres.append(l[6] + ' ' + '(' + l[7] + ')')

        lista_traduccion.append(dicc[l[9]])

    return lista_bolleanos, lista_nombres, lista_traduccion


Declaracion de todos los datos que voy a utilizar para el nuevo archivo. 
* La variable de arch_traducciones me va a servir para abrir el archivo donde tengo el diccionario de los nombres traducidos
* La variable del encabezado estan los datos que tiene que tener el archivo resultante
* las 3 listas auxiliares son para guardame los datos que necesito del archivo original para el filtrado. 

In [8]:
arch_traducciones = os.path.join(os.getcwd(), 'dicc_nombres.json')
encabezado = ['Year', 'Volcanic Explosivity Index', 'Volcano Type', 'Flag Tsunami', 'Flag Earthquake', 'Name and country']
aux = []
aux2 = []
aux3 = []

Declaro las variables necesarias para cargar el diccionario del archivo a una variable local, abrir el archivo de volcanes original y crear el nuevo. Cada uno con las rutas correctas para su acceso, asi funcionan en cualquier sistema operativo

In [9]:
with open(arch_traducciones, 'r', encoding='utf-8') as f:
    dicc_nombres = json.load(f)
archivo_volcan = os.path.join(os.getcwd(), 'significant-volcanic-eruption-database.csv')
archivo_volcan_nuevo = os.path.join(os.getcwd(), 'archivo_volcanes_nuevo.csv');

En este codigo lo que hice fue lo siguiente:
* Abri los archivos con with, asi me olvido de cerrarlos y es mas legible el codigo
* Utilizo la funcion seek() para volver el puntero del reader al inicio, asi puedo recorrerlo una vez mas para escrbir el archivo nuevo
* Escribo con el writer el encabezado que defini mas atras
* Haciendo uso de la funcion zip que me permite iterar sobre varios iterable a la vez, recorro todas las listas que hice anteriormente para escribir correctamente todas las filas del archivo nuevo, con la informacion que procese en la funcion anteriormente.

In [10]:
with open(archivo_volcan, 'r', encoding='utf-8') as volcanes:
    reader = csv.reader(volcanes, delimiter=';')
    header = next(reader)
    with open(archivo_volcan_nuevo, 'w', newline = '') as nuevo:
        writer = csv.writer(nuevo, delimiter = ';')
        generarNuevo(reader,aux,aux2,aux3,dicc_nombres)
        volcanes.seek(0)
        writer.writerow(encabezado)
        next(reader)
        for elem in zip(aux, aux2, aux3, reader):
            writer.writerow([elem[3][0],elem[3][11] if elem[3][11]!='' 
                                else '??',elem[2], elem[0][0] if elem[0][0] != '' else '??', elem[0][1] if 
                                elem[0][1] != '' else '??', elem[1]])